# Introduction/Business Problem

A business owner is looking to open a Tex-Mex restaurant in Seattle or one of its accompanying suburbs. Which area of the city is best to open a their restaurant?

# Data Needed

1. List of neighborhoods and their coordinates near Seattle
2. Geocoordinates of neighborhoods near seattle
3. Restaurants located within each neighborhood

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [22]:
#Define URL
url = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Washington'
content = requests.get(url).content

#Parse Page on BS4
soup = BeautifulSoup(content, 'html.parser')

#Pull Out Table
table = soup.find('table')

#Parse Columns and Headers
data = []
headers = False
data = []

#Iterate Through Rows For Data
for row in table.find_all('tr'):
    header = row.find_all('th')
    header_values = [cell.text.strip() for cell in header]
    cells = row.find_all('td')
    row_values = [cell.text.strip() for cell in cells]
    
    if not headers:
        headers = header_values
    
    data.append(row_values)

,City,County
4,Algona,King
11,Bellevue,King
15,Black Diamond,King
24,Burien,King
27,Carnation,King
37,Clyde Hill,King
43,Covington,King
47,Des Moines,King
49,Duvall,King
61,Federal Way,King


In [23]:
#Create DF and Filter for King County
df = pd.DataFrame(data)

df = df.iloc[:,[0,2]]

df.columns = ['City', 'County']

df = df.loc[df['County'] == 'King']

In [39]:
#Assign Lat/Long to Cities

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myapplication')

lats = []
longs = []

for city in df['City']:
    location = geolocator.geocode(city + ' Washington')
    
    lats.append(location.raw['lat'])
    longs.append(location.raw['lon'])
    
df['Latitude'] = lats
df['Longitude'] = longs

df

,City,County,Latitude,Longitude
4,Algona,King,47.2784027,-122.2542238
11,Bellevue,King,47.6144219,-122.1923372
15,Black Diamond,King,47.3083078,-122.0037583
24,Burien,King,47.469918,-122.3485274
27,Carnation,King,47.6475134,-121.9129419
37,Clyde Hill,King,47.631569,-122.2181054
43,Covington,King,47.3657791,-122.100222
47,Des Moines,King,47.398829,-122.3099093
49,Duvall,King,47.7423219,-121.9856782
61,Federal Way,King,47.313494,-122.3393103


In [92]:
#Plot Cities on Map of Seattle

import folium

df = df.dropna()

seattle_plot = folium.Map(location = [47.5, -122.3], zoom_start = 9)
locations = df[['Latitude', 'Longitude']].values.tolist()

for location in locations:
    folium.Marker(location).add_to(seattle_plot)
    
seattle_plot

In [104]:
#Create Function to Search Venues in City

from pandas.io.json import json_normalize
import requests



def Get_Venues(city, lat, long):
    
    CLIENT_ID = 'UEDDKGFX01JR3XXXZCIVN1A2PRZSPEDOVE5GOBPQUUYZTI5S'
    CLIENT_SECRET = '0SI5EIH22KPFV4KNLVYOPUL255XLGFIDT1PKYOZWW23KG4RB'
    VERSION = '20180604'

    LIMIT = 1000
    RADIUS = 10000

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        long, 
        RADIUS, 
        LIMIT,
        '4bf58dd8d48988d1c1941735')
    
    results = requests.get(url).json()
    
    items = results['response']['groups'][0]['items']

    
    ex = [item['venue']['categories'][0]['name'] for item in items]
    
    return ex.count('Mexican Restaurant')
    





In [107]:
#Search Number of Mexican Restaurants in Each City to Find City Missing Mexican

Mex_Counts = []

for i in range(len(df)):
    City = df['City'].iloc[i]
    Lat = df['Latitude'].iloc[i]
    Long = df['Longitude'].iloc[i]
    
    Mex_Count = Get_Venues(City, Lat, Long)
    
    Mex_Counts.append(Mex_Count)
    
    
df['Mexican Count'] = Mex_Counts


                 City County    Latitude            Longitude  Mexican Count
4              Algona   King  47.2784027         -122.2542238             63
11           Bellevue   King  47.6144219         -122.1923372             88
15      Black Diamond   King  47.3083078         -122.0037583             12
24             Burien   King   47.469918         -122.3485274             93
27          Carnation   King  47.6475134         -121.9129419              2
37         Clyde Hill   King   47.631569         -122.2181054             83
43          Covington   King  47.3657791          -122.100222             32
47         Des Moines   King   47.398829         -122.3099093             93
49             Duvall   King  47.7423219         -121.9856782              4
61        Federal Way   King   47.313494         -122.3393103             71
77           Issaquah   King  47.5348778         -122.0432974             18
81            Kenmore   King  47.7573202         -122.2440148             93

In [108]:
# Clean DF and sort to cities with minumum number of Mexican Restaurants

df = df[['City', 'Mexican Count']]

df = df.sort_values(by=['Mexican Count'])

print(df)

                 City  Mexican Count
27          Carnation              2
49             Duvall              4
125        North Bend              6
172        Snoqualmie              7
15      Black Diamond             12
100      Maple Valley             16
77           Issaquah             18
43          Covington             32
163         Sammamish             38
4              Algona             63
208       Woodinville             63
152           Redmond             68
61        Federal Way             71
153            Renton             77
121         Newcastle             78
105            Medina             81
37         Clyde Hill             83
165           Seattle             85
11           Bellevue             88
106     Mercer Island             88
170         Shoreline             92
124     Normandy Park             92
24             Burien             93
81            Kenmore             93
85           Kirkland             93
83               Kent             93
4

# Results


Overall, the results were varying with some cities in Seattle showing a very large amount of Mexican restaurants will a few pockets remaining for opportunity.


# Future Recomendations

In the future, other variables could be used to increase accuracry. Population and demographic information could help explain why certain cities have a large/minimal amount of restaurants. It also would be helpful to find specific neighborhoods within cities that are high population that are missing certain restaurant types.


# Conclusion

Based on the above results, Carnation is the best city to open a Mexican restaurant. These results will be based on lowest competition alone. Duvall and North Bend are the next best options for opening a restaurant.